# Chapter 3. word2vec

## 3.1 추론 기반 기법과 신경망


### 3.1.1 통계 기반 기법의 문제점

대규모 말뭉치를 다룰 때 문제가 발생  
단 1회의 처리 (SVD 등)에 단어의 분산 표현: 배치 학습  
신경망이 한번에 소량 (미니배치)의 학습 샘플씩 반복해서 학습하며 가중치를 갱신  
게다가 여러 머신과 여러 GPU를 이용한 병렬 계산도 가능해져 학습 속도를 높일 수 있음  

*reference* 3.5.3 통계 기반 vs 추론 기반

### 3.1.2 추론 기반 기법 개요

모델로 신경망을 사용  
모델은 맥락 정보를 입력받아 (출현할 수 있는) 각 단어의 출현 확률을 출력  
학습의 결과로 단어의 분산 표현을 얻는 것  


### 3.1.3 신경망에서의 단어 처리

고정 길이의 벡터로 변환 - 원핫 표현  
